# Final (merged) database unification
By Stephen Larroque @ Coma Science Group, GIGA Research, University of Liege
Creation date: 2019-02-28
License: MIT
v1.0.3

DESCRIPTION:
Postprocessing and statistical analyses on the final merged database. This unifies in the sense that it summarizes the data from similar columns from different databases (eg, sedation info) into one column. This thus represent expert knowledge specific to the databases used by the CSG, hence this script is certainly not applicable (or at least not without modifications) to other databases.

INSTALL NOTE:
You need to pip install pandas before launching this script.
Tested on Python 2.7.15

USAGE:
Input the final merged/unified database (merged_fmp_steph_manon_sarah_dicom_ecg_reports.csv), resulting from using csg_datafusion_db_merger.ipynb.

TODO:
* Figures

In [ ]:
# Forcefully autoreload all python modules
%load_ext autoreload
%autoreload 2

In [ ]:
# AUX FUNCTIONS

import os, sys

cur_path = os.path.realpath('.')
sys.path.append(os.path.join(cur_path, 'csg_fileutil_libs'))  # for unidecode and cleanup_name, because it does not support relative paths (yet?)

import re

from csg_fileutil_libs.aux_funcs import save_df_as_csv, _tqdm, compute_best_diag, reorder_cols_df, find_columns_matching, cleanup_name, replace_buggy_accents, convert_to_datetype, df_drop_duplicated_index, df_to_unicode, df_to_unicode_fast, cleanup_name_df, df_literal_eval, compute_best_diag, df_unify, df_translate, df_filter_nan_str, concat_vals_unique, reorder_cols_df


In [ ]:
# Nice plots!
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
# PARAMETERS

# Unified database, not yet postprocessed
unified_csv = r'databases_output\merged_fmp_steph_manon_sarah_dicom_ecg_reports.csv'

# Output folder for generated csv files and figures
output_dir = r'databases_output'

# Hide null values in plots?
plot_hide_nan = True

In [ ]:
# Import the csv dbs as dataframes
import pandas as pd
import numpy as np

cf_unified = pd.read_csv(unified_csv, sep=';', low_memory=False).dropna(axis=0, how='all').fillna('')  # drop empty lines
cf_unified

## Diagnosis computation
Compute the diagnosis for each CRS-R subscore.

In [ ]:
def subscores_to_diag(subscores):
    """Convert subscores (format: S00000) into a diagnosis (eg, coma)
    Also detects impossible and missing scores"""
    # If length is below 7, then it's malformatted (6 subitems of CRS-R + 1 prepending character S to say it's a subscore)
    if len(subscores) != 7:
        return 'na'
    # split into characters and convert to integer
    s = list(subscores)
    for i, x in enumerate(s):
        try:
            s[i] = int(x)
        except Exception as exc:
            s[i] = -1

    # sanity check
    if s[1] > 4 or s[2] > 5 or s[3] > 6 or s[4] > 3 or s[5] > 2 or s[6] > 3:
        # Impossible scores, higher than the maximum value for at least one category
        return 'impossible'

    # diagnosis conversion from subscores
    diag = 'coma'
    # arousal = 0 -> coma
    if s[6] <= 0:
        return diag
    # arousal >= 1 -> uws
    if s[6] >= 1:
        diag = 'vs/uws'
    # communication >= 1 or verbal >= 3 or motor between 3 and 5 or visual between 2 and 5 or auditive/command == 3 -> mcs
    if s[5] >= 1 or s[4] >= 3 or 3 <= s[3] <= 5 or 2 <= s[2] <= 5 or s[1] == 3:
        diag = 'mcs-'
    # auditive/command == 3 or verbal == 3 or communication == 1 -> mcs+
    # mcs+ definition from Aubinet, C., Panda, R., Larroque, S. K., Cassol, H., Bahri, M. A., Carrière, M., ... & Thibaut, A. (2019). Reappearance of command-following is associated to recovery of language and consciousness networks: A longitudinal multiple-case report. Frontiers in Systems Neuroscience, 13, 8.
    if s[1] == 3 or s[4] == 3 or s[5] == 1:
        diag = 'mcs+'
    # auditive/command == 4 -> srmcs (systematically responsive mcs)
    # note this is a custom definition, it does not exist in the litterature, you can consider srmcs just like mcs+
    if s[1] == 4:
        diag = 'srmcs'
    # communication == 2 or motor == 6
    # note that we use a more lenient definition of emcs here, as one diagnosis is enough (ie, no need for two consecutive diagnosis of emcs)
    if s[5] == 2 or s[3] == 6:
        diag = 'emcs'
    return diag

# Main loop to convert all CRS-Rs subscores into diagnoses
debug = False
lastrowdatesubscores_prev = None
datesdiags_list = []
datesdiags_concat_list = []
diagmax_list = []
diagmin_list = []
diagorder_doc = ['coma', 'vs/uws', 'mcs', 'mcs-', 'mcs+', 'srmcs', 'emcs']

for idx, row in _tqdm(cf_unified.iterrows(), total=cf_unified.shape[0], desc='DIAG', unit='rows'):
    # Debug
    if debug:
        row = cf_unified.iloc[0]
    # Optimization: since rows might copy fields from other rows (because of merging process), if we stumble upon a line where the fields were copied, we can simply copy the diagnoses of previous row
    if lastrowdatesubscores_prev and row['fmpagg.CRSr::Date and subscores'] == lastrowdatesubscores_prev:
        datesdiags = datesdiags_list[-1]
        diagmax = diagmax_list[-1]
        diagmin = diagmin_list[-1]
    # Else we calculate diagnoses
    else:
        # Memorize current row
        lastrowdatesubscores_prev = row['fmpagg.CRSr::Date and subscores']
        # If dates and subscores field not empty
        if row['fmpagg.CRSr::Date and subscores'].strip() or row['sarah.crs_subscore'].strip():
            # Cleanup datessubscores
            datessubscores = []

            # Add CRS-R subscores from FMP db
            if row['fmpagg.CRSr::Date and subscores'].strip():
                # Evaluate as a literal (to transform string into a python object, eg a list)
                datessubscoreslit = df_literal_eval(row['fmpagg.CRSr::Date and subscores'])
                # If it's a list, process as a list of dates and subscores
                if isinstance(datessubscoreslit, list):
                    # Each date/subscores tuple is a string separated by a ':', we split on this
                    datessubscores = [x.split(':') for x in datessubscoreslit]
                    # Filter nan values
                    datessubscores = filter(lambda x: 'nan' not in x, datessubscores)
                # Else it's a string, a single date/subscores tuple
                else:
                    # It's a date/subscores tuple separated with ':'
                    if ':' in datessubscoreslit:
                        datessubscores = [datessubscoreslit.split(':')]
                    # There is no separation, probably it's a single subscores, with no date
                    else:
                        datessubscores = [['', datessubscoreslit]]

            # Also use Sarah's CRS-R subscores
            if row['sarah.crs_subscore'].strip():
                if '[' in row['sarah.crs_subscore']:
                    sarah_subscores = [['', x] for x in df_literal_eval(row['sarah.crs_subscore'])]
                else:
                    sarah_subscores = [['', row['sarah.crs_subscore']]]
                datessubscores.extend(sarah_subscores)

            # Debug
            if debug:
                print(row['fmpagg.CRSr::Date and subscores'].strip())
                print(df_literal_eval(row['fmpagg.CRSr::Date and subscores']))
                if row['sarah.crs_subscore'].strip():
                    print(sarah_subscores)
                print(datessubscores)

            # Transform from subscores to diagnoses
            datesdiags = [[date, subscores_to_diag(subscores)] for date, subscores in datessubscores]
            # Debug
            if debug:
                print(datesdiags)
            # Order diagnoses using Pandas discrete categories, so that we can easily grade the maximum and minimum diagnoses
            datesdiags_discrete = compute_best_diag(pd.Series([subscores for date, subscores in datesdiags]), diag_order=['', 'na', 'impossible'] + diagorder_doc + ['lis'], persubject=None)
            # Get max and min diagnoses
            diagmax = datesdiags_discrete.max()
            diagmin = datesdiags_discrete.min()
        else:
            # Else the dates-subscores field is empty, we will use other fields
            datesdiags = {}
            diagmax = ''
            diagmin = ''

        # Disambiguate non-doc patients using other fields (because the CRS-R cannot account for them, eg: LIS, controls)
        if 'no doc' in row['fmpagg.Final diagnosis'].lower():
            diagmax = 'no_doc'  # note: we separate this if from others because we want to keep this info only if we don't have any better info in other fields (because no_doc is quite vague)
        if 'lis incomplete' in row['fmpagg.CRSr::Best Diagnosis'].lower() or 'lis incomplete' in row['fmpagg.Final diagnosis'].lower():
            diagmax = 'lis_incomplete'
        elif 'lis' in row['fmpagg.CRSr::Best Diagnosis'].lower() or 'lis' in row['fmpagg.Final diagnosis'].lower() or 'lis' in row['manon.Diagnostic CRS-R at date of RMN'].lower() or 'lis' in row['manon.Diagnostic Final Comportemental'].lower():
            diagmax = 'lis'
        elif 'ctrl' in row['manon.Diagnostic CRS-R at date of RMN'].lower():
            diagmax = 'ctrl'

        # If empty, use other fields
        if diagmax in ['', 'na', 'impossible']:
            if row['manon.Diagnostic Final Comportemental'].strip():
                diagmax = diagmin = row['manon.Diagnostic Final Comportemental'].strip().lower()
            elif row['reports.final_diagnosis'].strip():
                diagmax = diagmin = row['reports.final_diagnosis'].strip().lower()
            elif row['steph.CRS-R on same day'].strip():
                diagmax = diagmin = row['steph.CRS-R on same day'].strip().lower()

    # Save current diagnoses
    datesdiags_list.append(datesdiags)
    datesdiags_concat_list.append([':'.join(d) for d in datesdiags])  # concatenate together dates and diagnoses to avoid nested lists (which would be very hard to decode from a string, particularly since the quotes can't be saved due to limitations in the csv engine limitations of Python standard library that is used by pandas)
    diagmax_list.append(diagmax)
    diagmin_list.append(diagmin)

    # Debug
    if debug:
        print(datesdiags)
        print(diagmin)
        print(diagmax)
        # TODO: store result
        # TODO: get min date and max date difference of CRS-R, and min date and max date between lowest diag and highest diag
        print(idx)
        break

# Print one of the results
diagmax_list

In [ ]:
# -- Additional calculations on dates
dates_dict = {'timeperiod': [], 'transitiontimefromlowestdiag': [], 'transitiontimefromfirst': [], 'transitiontimefromlatestlowestdiag': [], 'count': []}

datesdiags_prev = None
idx = -1
for datesdiags, diagmax, diagmin in _tqdm(zip(datesdiags_list, diagmax_list, diagmin_list), desc='DATECALC', unit='rows'):
    idx += 1
    if datesdiags_prev and datesdiags == datesdiags_prev:
        # Optimization: recopy if the previous field is exactly the same as current one
        for k in dates_dict.keys():
            dates_dict[k].append(dates_dict[k][-1])
    else:
        # Memorize current row
        datesdiags_prev = datesdiags

        if not datesdiags:
            # Empty
            dates_dict['timeperiod'].append('')
            dates_dict['transitiontimefromlowestdiag'].append('')
            dates_dict['transitiontimefromfirst'].append('')
            dates_dict['transitiontimefromlatestlowestdiag'].append('')
            dates_dict['count'].append(0)
        else:
            # Prepare dates and diagnoses as a DataFrame with dates as datetype
            cf_datesdiags = convert_to_datetype(pd.DataFrame(datesdiags, columns=['date', 'diag']), 'date', '%d/%m/%Y')
            #cf_dates = convert_to_datetype(pd.DataFrame([date for date, diag in datesdiags if date], columns=['date']), 'date', '%d/%m/%y')

            # Prepare the max and min dates of diagmax and diagmin
            diagmax_mindate = cf_datesdiags[cf_datesdiags['diag'] == diagmax]['date'].min()
            diagmin_mindate = cf_datesdiags[cf_datesdiags['diag'] == diagmin]['date'].min()
            diagmin_maxdate = cf_datesdiags[cf_datesdiags['diag'] == diagmin]['date'].max()
            # Calculate total time period covered by CRS-R assessments
            dates_dict['timeperiod'].append((cf_datesdiags['date'].max() - cf_datesdiags['date'].min()))
            # Calculate total transition time between first lowest CRS-R assessment and first highest CRS-R assessment
            if diagmax != diagmin and diagmax in diagorder_doc and diagmin in diagorder_doc and not pd.isnull(diagmax_mindate) and not pd.isnull(diagmin_mindate):
                dates_dict['transitiontimefromlowestdiag'].append(diagmax_mindate - diagmin_mindate)
            else:
                dates_dict['transitiontimefromlowestdiag'].append('')
            # Calculate total transition time between first CRS-R assessment ever and first highest CRS-R assessment
            if diagmax in diagorder_doc and not pd.isnull(diagmax_mindate):
                dates_dict['transitiontimefromfirst'].append(diagmax_mindate - cf_datesdiags['date'].min())
            else:
                dates_dict['transitiontimefromfirst'].append('')
            # Calculate total transition time between first highest CRS-R assessment and latest lowest CRS-R assessment
            if diagmax != diagmin and diagmax in diagorder_doc and diagmin in diagorder_doc and not pd.isnull(diagmax_mindate) and not pd.isnull(diagmin_maxdate):
                dates_dict['transitiontimefromlatestlowestdiag'].append(diagmax_mindate - diagmin_maxdate)
            else:
                dates_dict['transitiontimefromlatestlowestdiag'].append('')
            # Calculate the number of CRS-R assessments/diagnoses
            dates_dict['count'].append(len(datesdiags))
            
            # DEBUG: if you want to print the relative full subject record, just use the current counter index as the row iloc of the unified dataframe
            #print(cf_unified.iloc[idx])
            
            # IMPORTANT NOTE: about the diagmax in diagorder_doc test: useless to simplify values before running this, because anyway if a diagnosis is not in diagorder_doc, it means it was NOT derived from calculating the CRS-R but from other fields, thus there is no CRS-R and no assessment date, so no temporal info to calculate anyway

# Show result
dates_dict

In [ ]:
# Assemble back everything into the cf_unified and save as csv! Preserve columns order!
cf_unified['unified.diagnosis_best'] = diagmax_list
cf_unified['unified.diagnosis_worst'] = diagmin_list
cf_unified['unified.diagnoses_dates_list'] = datesdiags_concat_list
cf_unified['unified.diagnoses_timeperiod'] = dates_dict['timeperiod']
cf_unified['unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag'] = dates_dict['transitiontimefromlowestdiag']
cf_unified['unified.diagnoses_transitiontimefromfirstcrsrtofirsthighestdiag'] = dates_dict['transitiontimefromfirst']
cf_unified['unified.diagnoses_transitiontimefromlatestlowestdiagtofirsthighestdiag'] = dates_dict['transitiontimefromlatestlowestdiag']
cf_unified['unified.diagnoses_count'] = dates_dict['count']

# Save to CSV
cf_unified_unicode = df_to_unicode_fast(cf_unified)
out_db = unified_csv[:-4]+'_unifieddiag.csv'
if save_df_as_csv(cf_unified_unicode, out_db, fields_order=None, csv_order_by='name', blankna=False, date_format='%Y-%m-%d'):
    print('Merged database successfully saved in %s!' % (out_db))
else:
    print('ERROR: the database could not be saved!')
# Display result
cf_unified

## Columns unification
Unify the data from multiple similar columns (eg, sedation info is sometimes available in one column but not in the other).
This will make data more easily processable.
Also the sequential order specified for the unification bears an information of quality: we first retrieve the info from most reliable columns, then if missing we use the other less reliable columns.
In other words: not only unification makes it easier to process data, it also make it more reliable by prioritizing the source.

In [ ]:
# Unify sedation

# Most reliable is from MRI acquisition sheets
# Then from CSG MRI attendant database
# Then from the FileMakerPro database
cf_unified = df_unify(cf_unified, ['steph.MRI sedation', 'manon.sedation epi ', 'fmpagg.MRI::Sedation EPI'], 'unified.episedation')
# Then finally from a combination of the dicom path and patients reports, either one can be wrong but combined it's more reliable
for i, row in cf_unified[cf_unified['unified.episedation'].isnull()].iterrows():
    report_sedat = df_literal_eval(row['reports.mri_sedation'])
    # If they agree, then it's all fine
    if (isinstance(report_sedat, list) and row['fmpagg.Dicom Path Sedation'] in report_sedat) or (row['fmpagg.Dicom Path Sedation'] == report_sedat):
        cf_unified['unified.episedation'].iloc[i] = row['fmpagg.Dicom Path Sedation']
    # If not, then we write conflict because we cannot fix this automatically
    else:
        cf_unified['unified.episedation'].iloc[i] = 'conflict'

# Show result
cf_unified['unified.episedation'].value_counts()

In [ ]:
# Unify acute
# most reliable, using dicom acquisition date directly against injury date from filemakerpro database
# less reliable, using only data from reports
cf_unified = df_unify(cf_unified, ['fmpagg.AcuteDicom', 'reports.acute'], 'unified.acute')

# Unify gender
# most reliable, done by hand from clinical sheets
# then from filemakerpro database, mid reliability
# then from reports, should be reliable enough (based on pronouns "he", "she" in various languages)
cf_unified = df_unify(cf_unified, ['steph.gender', 'fmpagg.Sexe', 'reports.gender'], 'unified.gender')

# Unify age
cf_unified = df_unify(cf_unified, ['manon.Age', 'reports.age'], 'unified.age')

# Unify etiology
cf_unified = df_unify(cf_unified, ['fmpagg.Etiology',
                                   'fmpagg.Etiology specified.',
                                   'steph.etio (from PET, still need to look in acute folder and whole dossier_hospi folders and hospital anamnese on omnipro)',
                                   'manon.Aetiology',
                                   'reports.accident_etiology'], 'unified.etiology')
# Remove nan values
cf_unified['unified.etiology'] = df_filter_nan_str(cf_unified['unified.etiology'])
# Fill in "other" etiology from other columns, because probably more infos elsewhere
cf_unified.loc[cf_unified['unified.etiology'].isin(['other', 'hsa']), 'unified.etiology'] = cf_unified['fmpagg.Etiology specified.']

print('All unifications done!')

In [ ]:
# Remove nan values from unified columns (for easier use)
cf_unified['unified.episedation'] = df_filter_nan_str(cf_unified['unified.episedation'])
cf_unified['unified.etiology'] = df_filter_nan_str(cf_unified['unified.etiology'])
cf_unified['unified.gender'] = df_filter_nan_str(cf_unified['unified.gender'])

## Values simplification
Now that we unified some columns, we will simplify the values they contain, so that we don't carry thousands of different values that are in fact similar (eg, etiology trauma and concussion and TBI ...). We want to uniformize all these values so that we can easily fetch and process them (eg, traumatic).

In [ ]:
# Simplify diagnoses
mapping_diag = {'vs uws': 'vs/uws',
                'uws': 'vs/uws',
                'coma tour de salle': 'coma',
                'coma (tour de salle)': 'coma',
                'vs selon tour de salle': 'vs/uws',
                'mcs-?': 'mcs- (maybe)',
                '[mcs-, emcs]': 'conflict',
                'mort c\xc3\xa9r\xc3\xa9brale': 'braindead',
                'mort cerebrale': 'braindead',
                '[lis, mcs, lis, mcs]': 'conflict',
                '[lis, uws]': 'conflict',
                '[lis, mcs]': 'conflict',
                'lis mcs lis mcs': 'conflict',
                'lis uws': 'conflict',
                'lis mcs': 'conflict',
                'mcs- emcs': 'conflict'
               }
print('Unique diagnoses before simplification:')
print(cf_unified['unified.diagnosis_worst'].unique())
print(cf_unified['unified.diagnosis_best'].unique())

cf_unified = df_translate(cf_unified, 'unified.diagnosis_best', mapping_diag, cleanup=False)
cf_unified = df_translate(cf_unified, 'unified.diagnosis_worst', mapping_diag, cleanup=False)

print('\nUnique diagnoses after simplification:')
print(cf_unified['unified.diagnosis_worst'].unique())
print(cf_unified['unified.diagnosis_best'].unique())

In [ ]:
# Simplify etiology
# Rename the multitude of etiologies descriptions into a few, to better plot
mapping_etio = {
    'traumatiques accident de la circulation accident du travail chute violence et autres': 'traumatic',
    'pas traumatiques - anoxie ex arrt cardiaque noyade pendaison intoxication co ranimation': 'nt anoxic',
    'pas traumatiques avc ex hmorragie infarctus aneurysme ischmie hypertension': 'nt stroke',
    'pas traumatiques - infection encphalite': 'nt infection',
    'pas traumatiques - mtaboliques ex hypoglycmie hyperglycmie': 'nt metabolic',
    'pas traumatiques - autre intoxication': 'intoxication',
    'pas traumatiques - autre': 'other',
    'ischemic stroke': 'nt stroke - ischemic',
    'tbi': 'traumatic',
    'hemorrhagic stroke': 'nt stroke - hemorrhage',
    'anoxia': 'nt anoxic',
    'mix trau anox': 'mixte trauma anoxia',
    'avc': 'nt stroke',
    'arca': 'nt anoxic',
    'anoxic': 'nt anoxic',
    'arca sur infarctus': 'nt anoxic',
    'arrt respiratoire': 'nt anoxic',
    'hematoma': 'nt stroke - hemorrhage',
    'anevrysm': 'nt stroke - hemorrhage',
    'mix traumatic anoxic': 'mixte trauma anoxia',
    'mixte trauma anoxie': 'mixte trauma anoxia',
    'arca coronarien - ranim': 'nt anoxic',
    'pas traumatiques - tumeur crbrale': 'nt tumor',
    'pas traumatiques - autre epilepsie': 'nt epilepsia',
    'mix other': 'other',
    'nt anoxia': 'nt anoxic',
    'mixed traumatic anoxic': 'mixte trauma anoxia',
    'pas traumatiques - autre ams': 'nt ams',
    'infection': 'nt infection',
    'subarachnoid hemorrhage': 'nt stroke - hemorrhage',
    'hmatome capsulo-thalamique': 'nt stroke - hemorrhage',
    'infarctus ischmique': 'nt stroke - ischemic',
    'nt intoxication': 'intoxication',
    'MIX (ARCA+ANOX) ("encephalopathie post anoxique sur arret cardiaque respiratoire")': 'nt anoxic',
    'AVC HEMO': 'nt stroke - hemorrhage',
    'AVC hemo': 'nt stroke - hemorrhage',
    'MIX ARCA-HYPOXIA ("ARCA asystolie-hypoxie")': 'nt anoxic',
    'ARCA (embolie/thrombose c\xc3\xa9r\xc3\xa9brale)': 'nt anoxic',
    'ARCA': 'nt anoxic',
    'TBI': 'traumatic',
    'ANOX': 'nt anoxic',
    'rupture d\'an\xc3\xa9vrysme': 'nt stroke - hemorrhage',
    'MIX (ARCA+HYPOXIA)': 'nt anoxic',
    'AVC': 'nt stroke',
    'METABOLIC (TS insuline)': 'nt metabolic',
    'DRUGS (hematome sous dural aigu dans un contexte de prise de sintrom (anti-coagulant))': 'nt stroke - hemorrhage',
    'ARCA (cardiac arrest)': 'nt anoxic',
    'TBI (=AVP)': 'traumatic',
    'unknown': 'other',
    'Toxoplasmose c\xc3\xa9r\xc3\xa9brale (had HIV)': 'nt infection',
    'METABOLIC (insulin overdose)': 'nt metabolic',
    'AVC ischemic': 'nt stroke - ischemic',
    'HEMO sur rupture d\'anevrysme': 'nt stroke - hemorrhage',
    'MENINGITE': 'nt infection',
    'EPILEPSIA (crise comitiale dans sevrage \xc3\xa9thylique, h\xc3\xa9matome sous et extradural)': 'nt epilepsia',
    'Metabolic? "TS \xc3\xa0 l\'insuline"': 'nt metabolic',
    'ANOX+ARCA (secondary)': 'nt anoxic',
    'ARCA (ANOX)': 'nt anoxic',
    'Hypoxia': 'nt anoxic',
    'Hemo': 'nt stroke - hemorrhage',
    'HEMO': 'nt stroke - hemorrhage',
    'embolisation d\'anevrisme cerebral': 'nt stroke - hemorrhage',
    'HEMO sur anevrysme': 'nt stroke - hemorrhage',
    'rupture d\'an\xc3\xa9vrisme': 'nt stroke - hemorrhage',
    'anevrysme': 'nt stroke - hemorrhage',
    'ACR (ARCA?)': 'nt anoxic',
    'encephalopathie d\'origine indeterminee': 'nt infection',
    'encephalopathie post-traumatique': 'traumatic',
    'd\xc3\xa9faillance multi organique sur choc septique et cardiog\xc3\xa9nique': 'nt infection',
    'TBI Chute (dans un \xc3\xa9tat de sevrage?)': 'traumatic',
    'AVP': 'traumatic',
    'hematome sous dural': 'nt stroke - hemorrhage',
    'post anoxie (arret cardio-respiratoire': 'nt anoxic',
    'Pas traumatiques AVC (ex. h\xc3\xa9morragie, infarctus, aneurysme, isch\xc3\xa9mie, hypertension)': 'nt stroke',
    'Hypoglycemie': 'nt metabolic',
    'Hematoma': 'nt stroke - hemorrhage',
    "['Trauma', 'TBI', 'Trauma', 'TBI']": 'traumatic',
    "['Trauma', 'TBI']": 'traumatic',
    'Trauma': 'traumatic',
    'TBI+ARCA': 'mixte trauma anoxia',
    'Anoxie': 'nt anoxic',
    'Pneumococcal meningo-encephalitis': 'nt infection',
    'Enc\xc3\xa9phalite avec Anoxie': 'nt infection',
    "['Ischemic stroke', 'NTBI Thrombose art\\xc3\\xa8re basilaire']": 'nt stroke - ischemic',
    'Recidive AVC': 'nt stroke',
    'ARCA ': 'nt anoxic',
    "['TBI accident moto', 'TBI', 'TBI accident moto', 'TBI']": 'traumatic',
    'TBI accident moto': 'traumatic',
    "['TBI accident moto', 'TBI']": 'traumatic',
    'TBI AVP': 'traumatic',
    'H\xc3\xa9morragie sous arachnoidienne': 'nt stroke - hemorrhage',
    'H\xc3\xa9morragie': 'nt stroke - hemorrhage',
    'ARCA sur h\xc3\xa9morragie et choc hypovol\xc3\xa9mique': 'nt stroke - hemorrhage',
    "['ARCA sur h\\xc3\\xa9morragie et choc hypovol\\xc3\\xa9mique', 'ARCA', 'ARCA sur h\\xc3\\xa9morragie et choc hypovol\\xc3\\xa9mique', 'ARCA']": 'nt stroke - hemorrhage',
    "['ARCA sur h\\xc3\\xa9morragie et choc hypovol\\xc3\\xa9mique', 'ARCA']": 'nt stroke - hemorrhage',
    'intoxication': 'nt intoxication',
    'Hemorragie': 'nt stroke - hemorrhage',
    'anoxie': 'nt anoxic',
    "['anoxie', 'anoxie', 'ARCA', 'ARCA']": 'nt anoxic',
    "['anoxie', 'ARCA']": 'nt anoxic',
    'NTBI AVC  ': 'nt stroke',
    'hypoxie': 'nt anoxic',
    'Pas traumatiques - anoxie (ex. arr\xc3\xaat cardiaque, noyade, pendaison, intoxication CO, r\xc3\xa9animation)': 'nt anoxic',
    'hydroc\xc3\xa9phalie obstructive due \xc3\xa0 son aspect (contexte septique)': 'nt infection',
    'Traumatiques (accident de la circulation, accident du travail, chute, violence et autres) ': 'traumatic',
    'Pas traumatiques-anoxie (ex.arret cardiaque, noyade, pendaison, intoxication CO, r\xc3\xa8animation)': 'nt anoxic',
    'Pas traumatiques - infection (enc\xc3\xa9phalite)': 'nt infection',
    'Anevrisme': 'nt stroke - hemorrhage',
    'Pas traumatiques - m\xc3\xa9taboliques (ex. hypoglyc\xc3\xa9mie, hyperglyc\xc3\xa9mie)': 'nt metabolic',
    'Encephalopathie': 'nt infection',
    'Embolie pulmonaire': 'nt infection',
    'NTBI ARCA ': 'nt anoxic',
    'Mixte (ARCA + trauma)': 'mixte trauma anoxia',
    'Asphyxie + ARCA (coma post-anoxique)': 'nt anoxic',
    'AVP/TC': 'traumatic',
    'metabolic': 'nt metabolic',
    'Mixte (trauma + anoxie)': 'mixte trauma anoxia',
    'stroke - hemorrhage': 'nt stroke - hemorrhage',
    'Pas traumatiques - autre (epilepsie)': 'nt epilepsia',
    "post hemorragie (rupture d'anevrisme": 'nt stroke - hemorrhage',
    'Pas traumatiques - autre (AMS)': 'nt other',
    'Pas traumatiques - autre': 'nt other',
    "['other']": 'other',
    "['stroke - hemorrhage']": 'nt stroke - hemorrhage',
    'post trauma (le': 'traumatic',
    "['Pas traumatiques - anoxie (ex. arr\\xc3\\xaat cardiaque', 'noyade', 'pendaison', 'intoxication CO', 'r\\xc3\\xa9animation)']": 'nt anoxic',
    'infarctus ischa c mique': 'nt stroke - ischemic',
    'mixed other': 'other',
    'intoxication': 'nt intoxication',
    'Hypothermia': 'nt metabolic',
    'Anurysm': 'nt stroke - hemorrhage',
    'h\xc3\xa9morragie c\xc3\xa9r\xc3\xa9brale ': 'nt stroke - hemorrhage',
    'Pas traumatiques - tumeur c\xc3\xa9r\xc3\xa9brale': 'nt tumor',
    "post traumatisme (chute lors d'un malaise,": 'traumatic',
    'post traumatisme cranien (avp': 'traumatic',
    'NTBI TS insuline': 'nt metabolic',
    'Pas traumatiques - autre (intoxication)': 'nt intoxication',
    'arraat respiratoire': 'nt anoxic',
    'Enc\xc3\xa9phalopathie h\xc3\xa9patique': 'nt infection',
    'post hemorragie cerebrale par embolisation (le': 'traumatic',
    'Post neurochirurgie': 'other',
    "['post encephalopathie anoxique apres arret cardio-respiratoire (le']": 'nt anoxic',
    'Not estabilished, probably attempted suicide with insuline': 'nt metabolic',
    "['traumatic']": 'traumatic',
    'ARCA Coronarien - r\xc3\xa9anim\xc3\xa9': 'nt anoxic',
    'Epilepsy & Toxoplasmose': 'nt infection', 'H\xc3\xa9matome capsulo-thalamique': 'nt stroke - hemorrhage',
    'marlurg multipe sclerosis': 'nt metabolic',
    'H\xc3\xa9morragie ponto-p\xc3\xa9donculaire': 'nt stroke - hemorrhage',
    "['H\\xc3\\xa9morragie ponto-p\\xc3\\xa9donculaire', 'Hemorragie']": 'nt stroke - hemorrhage',
    'trauma': 'traumatic',
    "['post- traumatique peut alors faire suite a cette experience (sigalovsky', '', 'post-infectieuses severes (carroll et mastaglia', '']": 'traumatic',
    "['post-infectieuses severes (carroll et mastaglia', '', 'post-infectieuses severes (carroll et mastaglia', '', 'post-infectieuses severes (carroll et mastaglia', '', 'post-comateux conscients presentaient une composante p']": 'traumatic',
    '[\'traumatisme cra^ nien\', \'le lis peut e^ tre\', "accident lis (rang) base de donne\' es de l\'alis", \'post-infectieuses severes (carroll et mastaglia\', \'\']': 'traumatic',
    'trauma rehabil': 'traumatic',
    'arret cardiaque (le': 'nt anoxic',
    'traumatises craniens severes presentant une alteration prolongee de la conscience est estimee entre': 'traumatic',
    'trauma rehabil,': 'traumatic',
    "['traumatique-avc lg 52 ans m aphasie', 'trauma c ta 63 ans f aphasie']": 'traumatic',
    'post--brainstem stroke (': 'nt stroke - hemorrhage',
    'Hemorragie du TC': 'nt stroke - hemorrhage',
    'traumatisme cranien (avril': 'traumatic',
    'trauma rl-imbll': 'traumatic',
    'trauma (': 'traumatic',
    'post-hematome temporo-parietal droit (': 'nt stroke - hemorrhage',
    'months post-tbi (date of accident:': 'traumatic',
    "['Ischemic stroke', 'NTBI  AVC du tronc c\\xc3\\xa9r\\xc3\\xa9bral']": 'nt stroke - ischemic',
    'post-hypoglycemie (': 'nt metabolic',
    'post-arca (': 'nt anoxic',
    'arrest, absence of pupillary responses within days': 'nt anoxic',
    "post hemorragie meningee sur rupture d'anevrisme sylvien droit (": 'nt stroke - hemorrhage',
    'arrest': 'nt anoxic',
    'post anoxie (le': 'nt anoxic',
    'post-trauma (accident de motocyclette sans port de casque': 'traumatic',
    'months after a traumatic brain injury which had occurred in december': 'traumatic',
    'post-- neurochirurgie (': 'other',
    'post-hematome parietal (': 'nt stroke - hemorrhage',
    'post-traumatique (': 'traumatic',
    'traumatic brain injury receiving amantadine daily for 4 weeks [': 'traumatic',
    'post-traumatic epilepsy (gaiyayzis, et al,': 'traumatic',
    "traumatic |male |sonde ventriculaire a voie d'entree |": 'traumatic',
    'accident =': 'traumatic',
    'post-trauma (accident de snowboard en decembre': 'traumatic',
    'post-anoxique (': 'nt anoxic', 'trauma et encephalopathie anoxique (': 'traumatic',
    'post arret cardiaque (a confirmer) (le': 'nt anoxic',
    'cardiac arrest': 'nt anoxic',
    "Rupture d'an\xc3\xa9vrysme": 'nt stroke - hemorrhage',
    'months in england) or more than one year for a traumatic etiology, he is in a permanent vegetative state [pic](jennett and plum': 'traumatic',
    
    # Errors of reports extraction, we blank them
    'months drawn from 1,': '',
    'years later, gowers (': '',
    'traumatique cause non-traumatique |': '',
    "traumatique et 49 patients dont la cause est non traumatique) ont ete evalues grace a l'echelle de recuperation du coma (crs-r) (3) a 1, 3,": '',
    "accident lis (rang) base de donne' es de l'alis": '',
    'traumatique et de 26 % pour les non traumatiques (bruno, gosseries, vanhaudenhuyse, chatelle & laureys,': '',
    "['post-anoxique ? revue medicale de liege', 'post-infectieuses severes (carroll and mastaglia', 'post-anoxique ? revue medicale de liege']": '',
    "['traumatique cause non-traumatique |', 'post-anoxique ? revue medicale de liege', 'post-anoxique ? revue medicale de liege', 'traumatique non-traumatique |', 'post-anoxique ? revue medicale de liege', 'traumatique non-traumatique |']": '',
    'post-operatoires chez les nouveaux-nes entre': '',
    "['traumatic brain injury predict poor outcome? neuropsychol rehabil', '', 'post-traumatic vegetative states:']": '',
    'traumatic and nontrau- patient': '',
    'months ons in 1st group; and at baseline,': '',
    "['accident | |(% total) | | |s | |lis (rang) | | | |base de donnees |']": '',
    '[\'post-infectieuses severes (carroll and mastaglia\', \'trauma rehabil\', \'\', \'trauma rehabil\', \'\', \'trauma\', "accident d\'anesthesie regionale (durrani and winnie"]': '',
    'post-comateux conscients presentaient une composante p': '',
    'months of ons; 6 before implantation and after': '',
    'years before (51% rr)': '',
    "['trauma |anoxia |anoxia |trauma |ischemia |brainstem stroke | |time of fmri (days after insult) |32 |7 |']": '',
    'trauma volume 19, number 7,': '',
    'months or longer (shewmon': '',
}

cf_unified = df_translate(cf_unified, 'unified.etiology', mapping_etio, cleanup=False)

cf_unified['unified.etiology'].unique()

In [ ]:
# Simplify gender
mapping_gender = {
    'Male': 'M',
    'H': 'M',
    '[\'H\']': 'M',
    'HMale': 'M',
    "['Female', 'F']": 'F',
    "['H', 'Male']": 'M',
    "['M', 'F']": '',
    "['M', 'F', 'M', 'M', 'F', 'M']": '',
    "['F', 'M']": '',
    'Female': 'F',
}
cf_unified = df_translate(cf_unified, 'unified.gender', mapping_gender, cleanup=False)
cf_unified['unified.gender'].value_counts()

In [ ]:
# Simplify sedation
from collections import OrderedDict
mapping_sedat = OrderedDict([
    ('\[\'no\'\]', 'no'),
    ('no \(curare\)', 'no'),
    ('no + curare', 'no'),
    ('no \(nimbex\)', 'no'),
    ("\['no \(nimbex\)', 'no', 'no \(nimbex\)', 'no'\]", 'no'),
    ('after all bold', 'no'),
    ('only for dti', 'no'),
    ('only for structural', 'no'),
    ("\['no\?', 'no', 'no\?', 'no'\]", 'no'),
    ('no \(esmerol - curare\)', 'no'),
    ('no +', 'both'),
    ('\'no\',', 'both'),
    (', \'no\'', 'both'),
    ('1no/2yes', 'both'),
    ('sevoflurane', 'yes'),
    ('isoflurane', 'yes'),
    ('propofol', 'yes'),
    ('diprivan', 'yes'),
    ('dormicum', 'yes'),
    ('dormicom', 'yes'),
    ('sevo', 'yes'),
    ('s\xc3\xa9voflurane', 'yes'),
    ('yes', 'yes'),
    ('sedated just for', 'no'),
    ('no', 'no'),
    ('\?', ''),
])
cf_unified['unified.episedationsimple'] = cf_unified['unified.episedation']
cf_unified = df_translate(cf_unified, 'unified.episedationsimple', mapping_sedat, cleanup=False, partial=True)
cf_unified['unified.episedationsimple'].value_counts()
cf_unified[['unified.episedation', 'unified.episedationsimple']].iloc[229]

## Saving final unified CSV
Save the DataFrame with unified columns back into a CSV file for later processing.

In [ ]:
# Reorder columns unified.* to put them first (just after the first 6 columns that represent the names and dates)
cf_unified = reorder_cols_df(cf_unified, cf_unified.columns.tolist()[:6] + find_columns_matching(cf_unified, 'unified', startswith=True))
# Save as CSV
cf_unified_unicode = df_to_unicode_fast(cf_unified)
out_db = unified_csv[:-4]+'_unifiedall.csv'
if save_df_as_csv(cf_unified_unicode, out_db, fields_order=False, csv_order_by='name', blankna=False, date_format='%Y-%m-%d'):
    print('Final unified database successfully saved in %s!' % (out_db))
else:
    print('ERROR: the database could not be saved!')

## Figures
A few temporary figures to explore the dataset we just constructed.

In [ ]:
# Figures on changes repartition from worst diagnosis to best
# take all patients with diagmin == vs/uws (or another diagnosis) and then calculate proportion of each diagmax
plot_hide_nan = False
cf_unified_perdiag = cf_unified.groupby('name')['unified.diagnosis_best', 'unified.diagnosis_worst'].agg(concat_vals_unique)
for diag in cf_unified['unified.diagnosis_worst'].unique():
    if not diag:
        continue
    fig = plt.figure()
    toplot = cf_unified_perdiag[cf_unified_perdiag['unified.diagnosis_worst'] == diag]['unified.diagnosis_best'].astype('str').value_counts(dropna=plot_hide_nan)
    toplot.plot(fig=fig, kind='pie', title='Best diagnosis change from worst being %s\n%i patients' % (diag.replace('?', 'maybe'), sum(toplot)), autopct='%.1f%%', figsize=(6,6))
    plt.axis('off')
    fig.savefig(os.path.join(output_dir, 'fig_diagchangefrom%s.png' % diag.replace('/', '').replace('?', '')), bbox_inches='tight', dpi=600)

# Save to CSV for closer inspection
cf_unified_perdiag_unicode = df_to_unicode_fast(cf_unified_perdiag.reset_index())
out_db = unified_csv[:-4]+'_diagchangefromworst.csv'
if save_df_as_csv(cf_unified_perdiag_unicode, out_db, fields_order=None, csv_order_by='name', blankna=False, date_format='%Y-%m-%d'):
    print('Database for closer inspection successfully saved in %s!' % (out_db))
else:
    print('ERROR: the database could not be saved!')


In [ ]:
# TODO: plot bestdiag repartition given number of CRS-R, first with 1 and then with max, and each between!
# To see if number of CRS-R is correlated with better diagnosis in general (but might be because we found these patients more interesting?)
# This is another way to represent results of Sarah. And on more subjects.


In [ ]:
# TODO: compute mean time and median time of unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag
# This is the mean/median time to reach the best diagnosis from first CRS-R
# and per diag?
cf_unified.loc[cf_unified['unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag'].isnull() | (cf_unified['unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag'] == ''), 'unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag'] = 0
cf_unified.loc[:, 'unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag'] = cf_unified['unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag'].apply(lambda x: x if isinstance(x, int) else x.days).astype('int')
cf_unified_time = cf_unified.loc[cf_unified['unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag'] > 0, 'unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag']
print(cf_unified_time.mean())
print(cf_unified_time.median())
fig = plt.figure()
cf_unified_time.plot(fig=fig, kind='hist', logx=True, bins=1000, title='Time to reach any best diagnosis')

for diag in cf_unified['unified.diagnosis_best'].unique():
    cf_unified_time_perdiag = cf_unified.loc[(cf_unified['unified.diagnosis_best'] == diag) & (cf_unified['unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag'] > 0), 'unified.diagnoses_transitiontimefromlowestdiagtofirsthighestdiag']
    print('Time to reach best diagnosis %s:' % diag)
    print('\t* Mean: %s' % cf_unified_time_perdiag.mean())
    print('\t* Median: %s' % cf_unified_time_perdiag.median())
    if len(cf_unified_time_perdiag) > 0:
        fig = plt.figure()
        cf_unified_time_perdiag.plot(fig=fig, kind='hist', logx=True, bins=1000, title='Time to reach best diagnosis %s' % diag)


In [ ]:
# TODO: Can also combine with accident date, to get the mean time of reaching each diagnosis (emcs, mcs, etc)